# TPD - Aula 4.2 - Regressão univariada

In [0]:
from pyspark.sql import SparkSession #importa a biblioteca que cria a seção do spark

In [0]:
#inicia a seção para a utilização do spark
spark = SparkSession.builder.appName("RegressaoLinear").getOrCreate() #cria a seção caso não exista ou obtém a já criada

In [0]:
%fs ls /FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/Dados_Azure,Dados_Azure,154,1654890939000
dbfs:/FileStore/tables/regressaoLinear.csv,regressaoLinear.csv,564,1656537294000


In [0]:
diretorioRegressao="/FileStore/tables/regressaoLinear.csv"  #diretório que contém o arquivo a ser utilizado

In [0]:
#definindo o esquema dos dados a serem lidos
from pyspark.sql.types import *  #import dos tipos
schema=StructType().add("X",IntegerType(),True).add("Y",StringType(),True)  #define o esquema a ser utilizado

In [0]:
#lendo arquivos armazenados CSV com o esquema definido
pagamentoSeguro = spark.read.format('csv').schema(schema).options(header='true',delimiter=';').load(diretorioRegressao)

In [0]:
pagamentoSeguro.printSchema()

root
 |-- X: integer (nullable = true)
 |-- Y: string (nullable = true)



In [0]:
pagamentoSeguro.show(5)

+---+-----+
|  X|    Y|
+---+-----+
|108|392,5|
| 19| 46,2|
| 13| 15,7|
|124|422,2|
| 40|119,4|
+---+-----+
only showing top 5 rows



In [0]:
import pyspark.sql.functions as F
pagamentoSeguro=pagamentoSeguro.select(F.col('X').alias("Apolices"), F.col('Y').alias("Valor_Pago"))  #adiciona nomes ao cabaçalho

In [0]:
pagamentoSeguro.show(5)

+--------+----------+
|Apolices|Valor_Pago|
+--------+----------+
|     108|     392,5|
|      19|      46,2|
|      13|      15,7|
|     124|     422,2|
|      40|     119,4|
+--------+----------+
only showing top 5 rows



In [0]:
pagamentoSeguroPonto=pagamentoSeguro.withColumn("Valor_Pago_Novo", F.regexp_replace(F.col("Valor_Pago"), "[,]", "."))  #troca o valor de "," para "."

In [0]:
pagamentoSeguroPonto.show(5)

+--------+----------+---------------+
|Apolices|Valor_Pago|Valor_Pago_Novo|
+--------+----------+---------------+
|     108|     392,5|          392.5|
|      19|      46,2|           46.2|
|      13|      15,7|           15.7|
|     124|     422,2|          422.2|
|      40|     119,4|          119.4|
+--------+----------+---------------+
only showing top 5 rows



In [0]:
#modificando o tipo string para numérico
pagamentoSeguroFinal=pagamentoSeguroPonto.select(F.col('Apolices'),F.col('Valor_Pago_Novo'), pagamentoSeguroPonto.Valor_Pago_Novo.cast('float').alias('Valor_Pago_Float'))

In [0]:
pagamentoSeguroFinal.show(5)

+--------+---------------+----------------+
|Apolices|Valor_Pago_Novo|Valor_Pago_Float|
+--------+---------------+----------------+
|     108|          392.5|           392.5|
|      19|           46.2|            46.2|
|      13|           15.7|            15.7|
|     124|          422.2|           422.2|
|      40|          119.4|           119.4|
+--------+---------------+----------------+
only showing top 5 rows



In [0]:
pagamentoSeguroFinal.printSchema()

root
 |-- Apolices: integer (nullable = true)
 |-- Valor_Pago_Novo: string (nullable = true)
 |-- Valor_Pago_Float: float (nullable = true)



Iniciando o Processo de Regressão

In [0]:
pagamentoSeguroFinal.describe().show()

+-------+------------------+-----------------+-----------------+
|summary|          Apolices|  Valor_Pago_Novo| Valor_Pago_Float|
+-------+------------------+-----------------+-----------------+
|  count|                63|               63|               63|
|   mean|22.904761904761905|98.18730158730159|98.18730196877131|
| stddev| 23.35194561605733|87.32755263404981|87.32755386147541|
|    min|                 0|                0|              0.0|
|    max|               124|             98.1|            422.2|
+-------+------------------+-----------------+-----------------+



In [0]:
#transformando os dados (linhas) em vetores
from pyspark.ml.feature import VectorAssembler  #importando a biblioteca responsável por criar o vetor a partir da coluna

assembler = VectorAssembler(inputCols=['Apolices'], outputCol='features')  #define o objeto para transformação
df_seguro = assembler.transform(pagamentoSeguroFinal) #aplica a transformação
df_seguro.printSchema()

root
 |-- Apolices: integer (nullable = true)
 |-- Valor_Pago_Novo: string (nullable = true)
 |-- Valor_Pago_Float: float (nullable = true)
 |-- features: vector (nullable = true)



In [0]:
df_seguro = df_seguro.select(['features','Valor_Pago_Float'])
df_seguro.show(5)

+--------+----------------+
|features|Valor_Pago_Float|
+--------+----------------+
| [108.0]|           392.5|
|  [19.0]|            46.2|
|  [13.0]|            15.7|
| [124.0]|           422.2|
|  [40.0]|           119.4|
+--------+----------------+
only showing top 5 rows



Criando o Modelo de Regressão

In [0]:
from pyspark.ml.regression import LinearRegression  #biblioteca que contém o modelo de regressão

lr = LinearRegression(maxIter=10, labelCol='Valor_Pago_Float') #define o objeto a ser utilizado para regressão
lrModel = lr.fit(df_seguro)

In [0]:
#Coeficientes angulares e lineares (a e b) da reta de regressão
print(f'Intercepto: {lrModel.intercept}\nCoeficiente Angular: {lrModel.coefficients.values}')

Intercepto: 19.994485035718697
Coeficiente Angular: [3.41382361]


In [0]:
#print das estatísticas do modelo
modelsummary = lrModel.summary

print("Variância Explicada:", modelsummary.explainedVariance)
print('R_2: ', modelsummary.r2)
print('Erro médio quadrático: ',modelsummary.meanSquaredError)

modelsummary.residuals.show(5)

Variância Explicada: 6254.310467068246
R_2:  0.8333466721037774
Erro médio quadrático:  1250.7419636077982
/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
+-------------------+
|          residuals|
+-------------------+
| 3.8125652674761454|
| -38.65713283055052|
| -48.67419213440233|
|-21.108600258352624|
| -37.14742784198987|
+-------------------+
only showing top 5 rows



Realizando a Previsão Através do Modelo

In [0]:
modelsummary.predictions.show(5)

+--------+------------------+------------------+
|features|  Valor_Pago_Float|        prediction|
+--------+------------------+------------------+
| [108.0]|             392.5|388.68743473252385|
|  [19.0]| 46.20000076293945| 84.85713359348998|
|  [13.0]|15.699999809265137| 64.37419194366747|
| [124.0]|422.20001220703125| 443.3086124653839|
|  [40.0]| 119.4000015258789|156.54742936786877|
+--------+------------------+------------------+
only showing top 5 rows

